# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [6]:
# Library untuk Dasar Python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Library untuk Pemodelan Random Forest dan Optuna sebagai metode Hyperparameter Tuning
import optuna
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Library untuk Evaluasi Model Klasifikasi
from sklearn.metrics import classification_report, confusion_matrix


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [7]:
data = pd.read_csv("Dataset_Inisiasi.csv").drop(columns='Unnamed: 0')
data.head()

,gradyear,gender,age,cute,shopping,Cluster
0,2007.0,Female,17.0,0.0,0.0,2
1,2007.0,Female,17.0,0.0,0.0,2
2,2008.0,Male,18.0,0.0,0.0,1
3,2006.0,Male,18.0,0.0,0.0,4
4,2006.0,Female,19.0,1.0,0.0,3


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [8]:
y = data["Cluster"]  # Mengubah y menjadi DataFrame agar kompatibel dengan OrdinalEncoder
X = data.drop(columns=["Cluster"])  # Masukkan variabel prediktor
X["gender"] = X["gender"].map({"Female": 0, "Male": 1})


# Membagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [9]:
# Membuat model
model1 = CatBoostClassifier(random_state=42, verbose=0)
model2 = LogisticRegression(random_state=42, max_iter=1000)

# Melatih model
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)


/home/anugerahsurya/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

Algoritma yang saya gunakan dalam pemodelan klasifikasi adalah CatboostClassifier. Hal ini dipilih berdasarkan baiknya kemampuan algoritma tersebut dalam kasus klasifikasi. Selain itu kemampuan algoritma ini dalam menyesuaikan data tanpa harus melalui proses hyperparameter tuning juga menjadi dasar dalam pemilihan algoritma ini. Kemampuan dalam Ordered Boosting juga menjadi keunggulan metode ini dalam menangani fitur numerik hingga kategorikan pada dataset. Dalam membandingkan performanya, saya juga akan menampilkan analisis dengan Regresi Logistik. Algoritma tersebut dijadikan pembanding karena memiliki pendekatan berbeda yaitu Boosting pada Catboost serta Linear dengan Transformasi Logit pada Regresi Logistik.

Sumber :
Dorogush, A. V., Ershov, V., & Gulin, A. (2018). CatBoost: gradient boosting with categorical features support. arXiv preprint arXiv:1810.11363.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [10]:
# Melakukan prediksi
y_pred = model1.predict(X_test)
y_pred2 = model2.predict(X_test)
y_tr = model1.predict(X_train)
y_tr2 = model2.predict(X_train)

print("===========Hasil Logistic Regression===========")
# Menampilkan Classification Report
print("Classification Report Data Training :")
print(classification_report(y_train, y_tr2))
print("Classification Report Data Testing :")
print(classification_report(y_test, y_pred2))

# Menampilkan Confusion Matrix
print("Confusion Matrix Data Training :")
print(confusion_matrix(y_train, y_tr2))
print("Confusion Matrix data Testing :")
print(confusion_matrix(y_test, y_pred2))

print("===========Hasil Catboost===========")
# Menampilkan Classification Report
print("Classification Report Data Training:")
print(classification_report(y_train, y_tr))
print("Classification Report Data Testing:")
print(classification_report(y_test, y_pred))

# Menampilkan Confusion Matrix
print("Confusion Matrix Data Training :")
print(confusion_matrix(y_train, y_tr))
print("Confusion Matrix Data Testing :")
print(confusion_matrix(y_test, y_pred))


===========Hasil Logistic Regression===========
Classification Report Data Training :
              precision    recall  f1-score   support

           1       1.00      0.99      0.99       830
           2       1.00      0.97      0.98      1782
           3       0.99      1.00      0.99      1987
           4       0.98      1.00      0.99       617
           5       0.98      1.00      0.99      1686

    accuracy                           0.99      6902
   macro avg       0.99      0.99      0.99      6902
weighted avg       0.99      0.99      0.99      6902

Classification Report Data Testing :
              precision    recall  f1-score   support

           1       1.00      0.99      0.99       207
           2       1.00      0.97      0.99       446
           3       0.99      1.00      0.99       497
           4       0.98      1.00      0.99       154
           5       0.99      1.00      0.99       422

    accuracy                           0.99      1726
   macro

Berdasarkan pemodelan dan evaluasi yang dilakukan, terlihat bahwa Kedua Model menunjukkan performa berbeda. Model Regresi Logistik menunjukkan akurasi yang cukup tinggi namun masih terdapat kesalahan prediksi pada class tertentu. Hal ini dapat diidentifikasi dari Confusion Matrix, dimana model tersebut cenderung memiliki kesalahan saat memprediksi cluster 5 namun diprediksi cluster 1. Hal tersebut menunjukkan model belum cukup baik dalam membedakan kondisi tersebut.

Berbeda dengan model Catboost yang cenderung memiliki model yang sempurna. Hal ini ditandai dengan ukuran evaluasi hingga 1. Selain itu pada Confusion Matrix yang diperoleh juga tidak terlihat nilai selain 0 pada bagian matriks selain diagonal. Hal tersebut menunjukkan keseluruh prediksi pada data testing benar.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [11]:
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# Fungsi objektif untuk tuning CatBoostClassifier
def objective_catboost(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10),
        'random_state': 42,
        'verbose': 0
    }
    
    model = CatBoostClassifier(**params)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()
    return score

# Fungsi objektif untuk tuning Logistic Regression
def objective_lr(trial):
    params = {
        'C': trial.suggest_float('C', 0.01, 10),
        'solver': 'liblinear',  # Support untuk L1 & L2
        'random_state': 42
    }
    
    model = LogisticRegression(**params)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()
    return score

# Optimasi dengan Optuna
study_catboost = optuna.create_study(direction='maximize')
study_catboost.optimize(objective_catboost, n_trials=20)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=20)

# Menampilkan hasil terbaik
print("Best CatBoost Parameters:", study_catboost.best_params)
print("Best Logistic Regression Parameters:", study_lr.best_params)

# Membuat model dengan hyperparameter terbaik
model3 = CatBoostClassifier(**study_catboost.best_params, random_state=42, verbose=0)
model4 = LogisticRegression(**study_lr.best_params, random_state=42)

# Melatih model
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)


[I 2025-02-22 18:14:55,071] A new study created in memory with name: no-name-7305964e-5a3e-49a7-b086-c9a0106ad470
[I 2025-02-22 18:15:28,339] Trial 0 finished with value: 1.0 and parameters: {'depth': 9, 'learning_rate': 0.2651684913427017, 'iterations': 986, 'l2_leaf_reg': 4.17547080007841}. Best is trial 0 with value: 1.0.
[I 2025-02-22 18:15:43,554] Trial 1 finished with value: 1.0 and parameters: {'depth': 7, 'learning_rate': 0.020916804559341216, 'iterations': 590, 'l2_leaf_reg': 5.477389237608564}. Best is trial 0 with value: 1.0.
[I 2025-02-22 18:16:00,142] Trial 2 finished with value: 1.0 and parameters: {'depth': 7, 'learning_rate': 0.293222047098217, 'iterations': 621, 'l2_leaf_reg': 5.9575264139094575}. Best is trial 0 with value: 1.0.
[I 2025-02-22 18:16:18,297] Trial 3 finished with value: 1.0 and parameters: {'depth': 6, 'learning_rate': 0.12986241264748577, 'iterations': 716, 'l2_leaf_reg': 2.889731301690729}. Best is trial 0 with value: 1.0.
[I 2025-02-22 18:16:28,909] 

Best CatBoost Parameters: {'depth': 9, 'learning_rate': 0.2651684913427017, 'iterations': 986, 'l2_leaf_reg': 4.17547080007841}
Best Logistic Regression Parameters: {'C': 3.2114200173328333}


/home/anugerahsurya/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=3.2114200173328333, random_state=42)

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [12]:
# Melakukan prediksi
y_pred3 = model3.predict(X_test)
y_pred4 = model4.predict(X_test)
y_tr3 = model3.predict(X_train)
y_tr4 = model4.predict(X_train)

print("===========Hasil Logistic Regression setelah Tuning===========")
# Menampilkan Classification Report
print("Classification Report Data Training :")
print(classification_report(y_train, y_tr4))
print("Classification Report Data Testing :")
print(classification_report(y_test, y_pred4))

# Menampilkan Confusion Matrix
print("Confusion Matrix Data Training :")
print(confusion_matrix(y_train, y_tr4))
print("Confusion Matrix data Testing :")
print(confusion_matrix(y_test, y_pred4))

print("===========Hasil Catboost setelah Tuning===========")
# Menampilkan Classification Report
print("Classification Report Data Training:")
print(classification_report(y_train, y_tr3))
print("Classification Report Data Testing:")
print(classification_report(y_test, y_pred3))

# Menampilkan Confusion Matrix
print("Confusion Matrix Data Training :")
print(confusion_matrix(y_train, y_tr3))
print("Confusion Matrix Data Testing :")
print(confusion_matrix(y_test, y_pred3))


===========Hasil Logistic Regression setelah Tuning===========
Classification Report Data Training :
              precision    recall  f1-score   support

           1       1.00      0.99      0.99       830
           2       1.00      0.97      0.98      1782
           3       0.99      1.00      0.99      1987
           4       0.98      1.00      0.99       617
           5       0.98      1.00      0.99      1686

    accuracy                           0.99      6902
   macro avg       0.99      0.99      0.99      6902
weighted avg       0.99      0.99      0.99      6902

Classification Report Data Testing :
              precision    recall  f1-score   support

           1       1.00      0.99      0.99       207
           2       1.00      0.97      0.99       446
           3       0.99      1.00      0.99       497
           4       0.98      1.00      0.99       154
           5       0.99      1.00      0.99       422

    accuracy                           0.99    

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

1. Berdasarkan proses hyperparameter tuning yang dilakukan terlihat bahwa tahapan tersebut tidak memberi peningkatan pada masing-masing algoritma dimana yang digunakan adalah Library Optuna menggunakan Algoritma Tree Parzen Structure (TPE).
Hasil tuning algoritma Logistic Regression tidak memberikan perubahan performa pada model yang dibangun. Hal ini dapat dilihat dari nilai F-1 Score pada masing-masing classnya. Selain itu pada model Catboost setelah dilakukan Tuning tidak ada perubahan karena model sudah mencapai kondisi optimal. Hal ini terjadi karena model berbasis tree cenderung bagus digunakan pada data skala besar dengan fitur yang cukup banyak, sehingga memberikan kemampuan algoritma untuk menetapkan decision yang tepat.

2. Berdasarkan tidak adanya peningkatan performa yang terjadi pada Logistic Regression setelah dituning, dapat diduga bahwa kondisi tersebut bisa terjadi karena parameter yang dituning tidak cukup baik saat mendefinisikan selang parameternya atau nilai hyperparameter yang didefinisikan belum cukup mengeksplorasi ruang yang lebih luas. Hal ini mengakibatkan nilai suatu parameter yang optimal tidak dicapai oleh algoritma. Selain itu karena pengujian tuning ini melibatkan Cross Validation dengan cv = 5, maka diperoleh bahwa model sudah cukup robust. Hal ini dapat dilihat dari performa saat tanpa tuning (Tanpa CV) dan setelah Tuning (dengan CV), performa model yang dibangun cenderung sama.
CV menggunakan rata-rata dari 5 split dataset train dan val untuk menjadi evaluasi dari suatu model setelah tuning, sehingga jika model sudah cukup robust, rataan tidak akan jauh berbeda dengan nilai saat pengujian tanpa CV.
Selain itu diperoleh juga pembuktian dari Algoritma Catboost yang tidak perlu dituning untuk mencapai kondisi optimalnya. Hal ini dapat dilihat setelah dituning dan diuji dengan CV diperoleh hasil yang sama dengan sebelumnya. Kondisi tersebut menjadi tanda bahwa model sudah berada pada kondisi optimal.
Dari kedua algoritma yang digunakan tidak ada indikasi overfitting, karena hasil pada train dan test tidak begitu berbeda signifikan. Hal ini dapat saja terjadi karena penggunaan CV, dimana train dan test didefinisikan ulang kembali.

3. Mengacu pada kelemahan untuk Logistic Regression dibanding CatboostClassifier, dapat diajukan tindakan untuk mengatasinya dengan cara eliminasi outlier. Hal ini karena metode Regresi Logistik cenderung linier walaupun diikuti dengan Transformasi Logit sehingga outlier pada data tentu dapat mempengaruhi hasil model secara signifikan. Selain itu melakukan transformasi untuk membuat distribusi data lebih baik, atau melakukan proses resampling (oversampling dan/atau undersampling) dapat menjadi opsi untuk meningkatkan keseimbangan pembelajaran yang dilakukan untuk masing-masing class. Namun hasilnya tetap sudah menunjukkan performa yang sangat baik, saat ditinjau dari F-1 Score mendekati 0.99 untuk setiap class.